## Imports

In [214]:
import gurobipy as gp
import json
import numpy as np
import os
import pandas as pd
import random
from gurobipy import GRB

## Read Data

In [216]:
TRANSFER_PERCENTAGE = 0
TRANSFER_PERCENTAGE = 0.25

if TRANSFER_PERCENTAGE > 0:
    INPUT_PATH = f"data/boston-logan/transfers/{TRANSFER_PERCENTAGE}"
else:
    INPUT_PATH = "data/boston-logan"

arrivals_df = pd.read_csv(os.path.join(INPUT_PATH, "arrivals.csv"))
departures_df = pd.read_csv(os.path.join(INPUT_PATH, "departures.csv"))
gates_df = pd.read_excel("data/boston-logan/Gates.xlsx")

if TRANSFER_PERCENTAGE > 0:
    with open(os.path.join(INPUT_PATH, "transfers.json")) as json_file:
        transfers = json.load(json_file)

all_flights_df = pd.concat([arrivals_df, departures_df])

In [217]:
arrivals_df

,Unnamed: 0,Flight Number,Arrival Time,Departure Time,Passengers,Type,final_passengers,percentage
0,0,2,25,136,106,Arrival,80,0.754717
1,1,3,41,161,147,Arrival,111,0.755102
2,2,6,73,187,187,Arrival,143,0.764706
3,3,7,77,188,121,Arrival,91,0.752066
4,4,10,111,233,94,Arrival,71,0.755319
...,...,...,...,...,...,...,...,...
63,63,120,1325,1447,127,Arrival,120,0.944882
64,64,124,1369,1499,88,Arrival,88,1.000000
65,65,126,1390,1513,240,Arrival,240,1.000000
66,66,127,1398,1513,87,Arrival,87,1.000000


In [218]:
gates_df ## Gates Start from 1 (Gate 0 is the terminal entry)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,53,45,43,31,29,25,23,19,9,3,15,17,21,23
1,1,53,0,4,10,22,24,28,30,34,44,50,62,64,68,70
2,2,45,4,0,6,18,20,24,26,30,40,46,58,60,64,66
3,3,43,10,6,0,12,14,18,20,24,34,40,52,54,58,60
4,4,31,22,18,12,0,2,6,8,12,22,28,40,42,46,48
5,5,29,24,20,14,2,0,4,6,10,20,26,38,40,44,46
6,6,25,28,24,18,6,4,0,2,6,16,22,34,36,40,42
7,7,23,30,26,20,8,6,2,0,4,14,20,32,34,38,40
8,8,19,34,30,24,12,10,6,4,0,10,16,28,30,34,36
9,9,9,44,40,34,22,20,16,14,10,0,6,18,20,24,26


## Utility Functions

In [219]:
def get_distance(i, j):
    return gates_df[i][j]

def get_arrival_time(i):
    result = all_flights_df[all_flights_df["Flight Number"] == i]["Arrival Time"]
    assert(len(result) == 1)
    return int(result)

def get_departure_time(i):
    result = all_flights_df[all_flights_df["Flight Number"] == i]["Departure Time"]
    assert(len(result) == 1)
    return int(result)

def get_num_passengers(i):
    result = all_flights_df[all_flights_df["Flight Number"] == i]["Passengers"]
    assert(len(result) == 1)
    return int(result)

### Tests

In [220]:
assert(get_distance(5,3) == 14)
assert(get_distance(14,14) == 0)
assert(get_distance(4,11) == 40)

In [221]:
## Basic Assertions on the Data
flight_numbers = sorted(list(arrivals_df["Flight Number"]) + list(departures_df["Flight Number"]))
assert(flight_numbers == list(range(1,131)))

flight_numbers = sorted(list(all_flights_df["Flight Number"]))
assert(flight_numbers == list(range(1,131)))

## Parameters

In [222]:
num_flights = 3
num_gates = 3

## Descriptors (for variables)

### Select Random Flights

In [247]:
np.random.seed(0)
num_flights = 5
F = [f"Flight_{i}" for i in [random.randint(1, 130) for j in range(num_flights)]]

In [248]:
F

['Flight_37', 'Flight_118', 'Flight_124', 'Flight_17', 'Flight_41']

### Select Fixed Flights

In [225]:
# num_flights = 3
# F = [f"Flight_{i}" for i in range(1, num_flights + 1)]
# F

In [226]:
G = [f"Gate_{i}" for i in range(1, num_gates + 1)]
G

['Gate_1', 'Gate_2', 'Gate_3']

In [227]:
flight_gate_dict = dict()
for f in F:
    for g in G:
        i = int(f.split("_")[1])
        j = int(g.split("_")[1])
        p = get_num_passengers(i)
        flight_gate_dict[(f,g)] = p

In [228]:
flight_gate_vars, passengers = gp.multidict(flight_gate_dict)

## Check Constraint 2

In [229]:
def check_constraints(x):
    for f1 in x.keys():
        i = int(f1.split("_")[1])
        ai, di = get_arrival_time(i), get_departure_time(i)
        for f2 in x.keys():
            j = int(f2.split("_")[1])
            if i <= j:
                continue
            aj, dj = get_arrival_time(j), get_departure_time(j)
            if (x[f1] == x[f2]) and (dj - ai) * (di - aj) > 0:
                return False
    return True

## Compute Objective Value

In [230]:
## Objective Value
def objective_value(x):
    objective = 0
    for f1 in x.keys():
        gate = x[f1]
        i = int(f1.split("_")[1])
        g = int(gate.split("_")[1])
        objective += get_num_passengers(i) * get_distance(0, g)

    if TRANSFER_PERCENTAGE > 0:
        print("Adding Transfer Distances to Objective Function...")
        for var_name_1 in x.keys():
            i = int(var_name_1.split("_")[1])
            gate_1 = x[var_name_1]
            k = int(gate_1.split("_")[1])
            for var_name_2 in x.keys():
                xjl = x[var_name_2]
                j = int(var_name_2.split("_")[1])
                gate_2 = x[var_name_2]
                l = int(gate_2.split("_")[1])

                i_key, j_key = str(i), str(j)
                if i_key in transfers:
                    if j_key in transfers[i_key]:
                        pij = transfers[i_key][j_key]
                        wkl = get_distance(k, l)

                        objective += pij * wkl
    return objective

## Brute Force Solution (Exact)

In [231]:
def get_assignments(i, n):
    if i == n:
        return [[]]
    assignments = []
    sub_assignments = get_assignments(i + 1, n)
    for gate in range(num_gates):
        for sub_assignment in sub_assignments:
            assignments.append(sub_assignment + [gate])
    return assignments

In [232]:
assignments = get_assignments(0, num_flights)

In [233]:
best_assignment, best_value = None, None
for i, assignment in enumerate(assignments):
    x = {F[i]: G[assignment[i]] for i in range(len(assignment))}
    if check_constraints(x):
        value = objective_value(x)
        if best_value == None or value < best_value:
            best_assignment = assignment
            best_value = value

print(best_assignment)
print(best_value)

Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Objective Function...
Adding Transfer Distances to Ob

In [234]:
print(len(assignments))

243


## Model

In [235]:
m = gp.Model('RAP')

### Design Variables

In [236]:
x = m.addVars(flight_gate_vars, name="assign")

### Constraint 1

In [237]:
constraint_1 = m.addConstrs((x.sum(f,'*') == 1 for f in F), name="constraint_1")

In [238]:
constraint_1

{'Flight_30': <gurobi.Constr *Awaiting Model Update*>,
 'Flight_15': <gurobi.Constr *Awaiting Model Update*>,
 'Flight_26': <gurobi.Constr *Awaiting Model Update*>,
 'Flight_68': <gurobi.Constr *Awaiting Model Update*>,
 'Flight_37': <gurobi.Constr *Awaiting Model Update*>}

### Constraint 2

In [239]:
for f1 in F:
    i = int(f1.split("_")[1])
    ai, di = get_arrival_time(i), get_departure_time(i)
    for f2 in F:
        j = int(f2.split("_")[1])
        if i <= j:
            continue
        aj, dj = get_arrival_time(j), get_departure_time(j)
        for g in G:
            name = f"{f1},{f2},{g}"
            const = m.addConstr((x[(f1, g)] * x[(f2, g)] * (dj - ai) * (di - aj)) <= 0, name=f"{f1},{f2},{g}")

### Objective Function

In [240]:
x

{('Flight_30', 'Gate_1'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_30', 'Gate_2'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_30', 'Gate_3'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_15', 'Gate_1'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_15', 'Gate_2'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_15', 'Gate_3'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_26', 'Gate_1'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_26', 'Gate_2'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_26', 'Gate_3'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_68', 'Gate_1'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_68', 'Gate_2'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_68', 'Gate_3'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_37', 'Gate_1'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_37', 'Gate_2'): <gurobi.Var *Awaiting Model Update*>,
 ('Flight_37', 'Gate_3'): <gurobi.Var *Awaiting Model Update*>}

In [241]:
objective = 0
for var_name in x.keys():
    xij = x[var_name]
    i = int(var_name[0].split("_")[1])
    j = int(var_name[1].split("_")[1])
    objective += xij * get_num_passengers(i) * get_distance(0, j)

if TRANSFER_PERCENTAGE > 0:
    print("Adding Transfer Distances to Objective Function...")
    for var_name_1 in x.keys():
        xik = x[var_name_1]
        i = int(var_name_1[0].split("_")[1])
        k = int(var_name_1[1].split("_")[1])
        for var_name_2 in x.keys():
            xjl = x[var_name_2]
            j = int(var_name_2[0].split("_")[1])
            l = int(var_name_2[1].split("_")[1])

            i_key, j_key = str(i), str(j)
            if i_key in transfers:
                if j_key in transfers[i_key]:
                    pij = transfers[i_key][j_key]
                    wkl = get_distance(k, l)

                    objective += pij * wkl * xik * xjl

Adding Transfer Distances to Objective Function...


In [242]:
transfers

{'2': {'1': 8, '4': 8, '5': 3, '8': 7},
 '3': {'1': 8, '4': 4, '5': 11, '8': 12, '28': 1},
 '6': {'1': 4,
  '4': 4,
  '5': 11,
  '8': 11,
  '9': 3,
  '12': 7,
  '17': 1,
  '19': 1,
  '28': 2},
 '7': {'1': 2, '4': 5, '9': 14, '12': 1, '20': 7, '38': 1},
 '10': {'4': 8, '9': 14, '12': 1},
 '11': {'4': 3,
  '5': 6,
  '12': 10,
  '17': 7,
  '19': 6,
  '20': 12,
  '21': 3,
  '26': 4},
 '13': {'12': 2, '17': 13, '19': 7},
 '14': {'12': 3,
  '17': 7,
  '19': 13,
  '20': 9,
  '23': 5,
  '26': 9,
  '27': 6,
  '28': 6},
 '15': {'12': 9, '17': 2, '19': 3, '21': 7, '22': 1, '26': 5},
 '16': {'12': 4, '21': 8, '22': 14, '26': 5, '27': 2},
 '18': {'12': 6,
  '19': 2,
  '21': 5,
  '22': 3,
  '26': 2,
  '27': 3,
  '40': 1,
  '47': 1},
 '24': {'21': 8,
  '23': 10,
  '26': 12,
  '27': 2,
  '28': 2,
  '31': 7,
  '36': 2,
  '38': 1,
  '40': 2},
 '25': {'21': 14,
  '22': 3,
  '23': 12,
  '26': 5,
  '27': 2,
  '28': 6,
  '31': 1,
  '35': 10,
  '38': 1,
  '45': 2},
 '29': {'23': 6,
  '26': 4,
  '27': 6,
  '2

In [243]:
m.setObjective(objective, GRB.MINIMIZE)

### Optimize

In [244]:
m.params.NonConvex = 2
m.params.MIPGap = 0.1
m.Params.TimeLimit = 600
m.optimize()

Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 15 columns and 15 nonzeros
Model fingerprint: 0x84c4b5ec
Model has 12 quadratic objective terms
Model has 30 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [7e+02, 3e+05]
  Objective range  [4e+03, 1e+04]
  QObjective range [4e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 3 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolved: 47 rows, 25 columns, 91 nonzeros
Presolved model has 21 bilinear constraint(s)
Variable types: 25 continuous, 0 integer (0 binary)

Root relaxation: objective 3.765000e+04, 5 iterations, 0.00 seconds (0.00 wo

### Inspect Results

In [245]:
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[Flight_30,Gate_2] 1.0
assign[Flight_15,Gate_2] 1.0
assign[Flight_26,Gate_3] 1.0
assign[Flight_68,Gate_3] 1.0
assign[Flight_37,Gate_3] 1.0
Total matching score:  37650.0


In [246]:
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[Flight_30,Gate_2] 1.0
assign[Flight_15,Gate_2] 1.0
assign[Flight_26,Gate_3] 1.0
assign[Flight_68,Gate_3] 1.0
assign[Flight_37,Gate_3] 1.0
Total matching score:  37650.0
